In [8]:
import torch
import torchani
from rdkit import Chem
from rdkit.Chem import AllChem


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
model = torchani.models.ANI2x(periodic_table_index=True).to(device)

In [17]:

class TorchANIEnergy:
    def __init__(self, model, smiles, batch_size=1):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model
        self.rdk_mol = Chem.MolFromSmiles(smiles)
        self.rdk_mol = Chem.AddHs(rdk_mol)
        AllChem.EmbedMolecule(rdk_mol)
        self.atomic_numbers = torch.tensor([atom.GetAtomicNum() for atom in self.rdk_mol.GetAtoms()]).to(device)
        self.data_ndim = 3 * len(self.atomic_numbers)
        self.batch_size = batch_size
        self.an_bs = self.atomic_numbers.unsqueeze(0).repeat(self.batch_size, 1)
                
    def get_energy(self, xyz):
        energies = self.model((self.an_bs, xyz)).energies
        return energies

smiles = 'CCO'
rdk_mol = Chem.MolFromSmiles(smiles)
rdk_mol = Chem.AddHs(rdk_mol)

In [18]:
energy = TorchANIEnergy(model, 'CCO', batch_size=1)

NameError: name 'get_atom_types_from_smiles' is not defined

In [11]:
energy.atomic_numbers

tensor([6, 6, 8, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [15]:
# generate random coordinates in the shape of (n_samples, n_atoms, 3)
xyz = torch.rand(1024, len(energy.atomic_numbers), 3).to(device)

In [16]:
import time
now = time.time()
energies = energy.get_energy(xyz)
print('Time:', time.time() - now)
print('Energies:', energies)

Time: 0.028456449508666992
Energies: tensor([-206.8431, -205.3601, -196.7456,  ..., -201.6755, -184.8934,
        -201.6019], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)


In [2]:
!pip install ogb

  Using cached ogb-1.3.6-py3-none-any.whl.metadata (6.2 kB)
  Using cached outdated-0.2.2-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached littleutils-0.2.2.tar.gz (6.6 kB)
  Preparing metadata (setup.py) ... done
Using cached ogb-1.3.6-py3-none-any.whl (78 kB)
Using cached outdated-0.2.2-py2.py3-none-any.whl (7.5 kB)
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=cc2ebed8331ed3aec5de718924d429404a6d2ac57785b36222a247afc7b00ec4
  Stored in directory: /home/radoslavralev/.cache/pip/wheels/6a/33/c4/0ef84d7f5568c2823e3d63a6e08988852fb9e4bc822034870a
Successfully built littleutils


In [ ]:
from ogb.utils import smiles2graph
graph = smiles2graph('O1C=C[C@H]([C@H]1O2)c3c2cc(OC)c4c3OC(=O)C5=C4CCC(=O)5')
graph

In [12]:
graph['num_nodes']

23

In [13]:
# create a (512, 23, 3) random tensor
pos = torch.rand(512, 23, 3).to(device)

In [14]:
graph['pos'] = pos

In [17]:
from models import MACEModel

ModuleNotFoundError: No module named 'models'